# 🛰️ Simple Building Height Inspector (Mataram)

Notebook ini dirancang untuk:
1.  **Ringan & Cepat**: Menggunakan estimasi luas untuk tinggi bangunan.
2.  **Sederhana**: Tanpa informasi pajak/detail rumit. Hanya visual.
3.  **Terfokus**: Filter bangunan berdasarkan ketinggian.

### Cara Jalanin:
1.  Klik menu **Runtime** > **Run all** (atau `Ctrl+F9`).
2.  Ikuti langkah Login Google Earth Engine jika diminta.
3.  Update **Latitude/Longitude** dan **Radius** di bawah peta untuk pindah lokasi.

In [ ]:
# @title 1. Install & Import Module
!pip install geemap
import geemap
import ee
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# @title 2. Authenticate Google Earth Engine
# @markdown Masukkan **Google Cloud Project ID** Anda di bawah ini (contoh: `mataram-sstb` atau `my-project-123`):
PROJECT_ID = "mataram-sstb" # @param {type:"string"}

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✅ Berhasil Connect ke GEE, Project: {PROJECT_ID}!")
except:
    print("🔄 Memulai Autentikasi... Ikuti langkah-langkah di browser.")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✅ Berhasil Login & Connect, Project: {PROJECT_ID}!")

In [ ]:
# @title 3. Konfigurasi Lokasi & Logika

# Lokasi Mataram
MATARAM_LAT = -8.5831
MATARAM_LON = 116.1165
ZOOM_LEVEL = 16

# --- WARNA BERDASARKAN TINGGI ---
# Format: (Min Height, Color)
COLORS = {
    'low':    '#22c55e', # Hijau (< 5m)
    'medium': '#eab308', # Kuning (5-10m)
    'high':   '#f97316', # Orange (10-15m)
    'sky':    '#ef4444'  # Merah (> 15m)
}

def estimate_height_from_area(area):
    """Rumus sederhana: Luas lebih besar = Bangunan lebih tinggi"""
    # Menggunakan ee.Number untuk perhitungan server-side GEE
    area = ee.Number(area)
    
    return ee.Algorithms.If(area.lt(80), 3.5,       # < 80m2 -> 3.5m
           ee.Algorithms.If(area.lt(150), 4.5,      # < 150m2 -> 4.5m
           ee.Algorithms.If(area.lt(250), 7.5,      # < 250m2 -> 7.5m
           ee.Algorithms.If(area.lt(400), 12.0,     # < 400m2 -> 12.0m
           18.0))))                                 # > 400m2 -> 18.0m

def add_height_property(feature):
    """Menambahkan property 'est_height' ke setiap fitur"""
    area = feature.geometry().area()
    height = estimate_height_from_area(area)
    return feature.set({'est_height': height, 'area_m2': area})

In [ ]:
# @title 4. Tampilkan Peta Interaktif (Lokasi & Radius Bisa Diubah)

# --- WIDGET CONTROL ---
style = {'description_width': 'initial'}
layout_full = widgets.Layout(width='98%')
layout_half = widgets.Layout(width='48%')

# 1. Height Slider
height_slider = widgets.FloatSlider(
    value=0, min=0, max=20.0, step=1.0,
    description='Tampilkan Tinggi > (m):',
    style=style, layout=layout_full
)

# 2. Location Inputs
lat_input = widgets.FloatText(
    value=MATARAM_LAT, 
    description='Latitude:', 
    style=style, layout=layout_half
)
lon_input = widgets.FloatText(
    value=MATARAM_LON, 
    description='Longitude:', 
    style=style, layout=layout_half
)

# 3. Radius Slider
radius_slider = widgets.FloatSlider(
    value=1.0, min=0.5, max=5.0, step=0.1,
    description='Radius Area (km):',
    style=style, layout=layout_full
)

# 4. Button Update
update_btn = widgets.Button(
    description='📍 Update Peta & Lokasi',
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_full,
    icon='map-marker'
)

# Container UI
ui_controls = widgets.VBox([
    widgets.HBox([lat_input, lon_input]),
    radius_slider,
    height_slider,
    update_btn
])

output_map = widgets.Output()

def update_map(b=None):
    with output_map:
        output_map.clear_output()
        print("⏳ Sedang memuat data baru...")
        
        # Get Values
        lat = lat_input.value
        lon = lon_input.value
        rad_km = radius_slider.value
        min_height = height_slider.value
        
        # Inisialisasi Peta
        m = geemap.Map(center=[lat, lon], zoom=15)
        m.add_basemap('SATELLITE')
        
        # Buat Geometri Lingkaran Baru
        roi = ee.Geometry.Point([lon, lat]).buffer(rad_km * 1000) # km to meters
        
        # Visualisasi Area Lingkaran (Outline Putih)
        m.addLayer(roi, {'color': 'white', 'fillColor': '00000000'}, 'Area Analisis')
        
        # Load Data
        dataset = ee.FeatureCollection("GOOGLE/Research/open-buildings/v3/polygons") \
            .filterBounds(roi) \
            .map(add_height_property) \
            .filter(ee.Filter.gte('est_height', min_height))
        
        # Style Function
        def style_function(feature):
            h = feature['properties']['est_height']
            color = COLORS['low']
            if h >= 15: color = COLORS['sky']
            elif h >= 10: color = COLORS['high']
            elif h >= 5: color = COLORS['medium']
            
            return {
                'fillColor': color,
                'color': color,
                'weight': 1,
                'fillOpacity': 0.6
            }

        # Konversi ke GeoJSON
        try:
            # Limit jika area terlalu besar untuk mencegah crash browser
            if rad_km > 3.0:
                print(f"⚠️ Radius besar ({rad_km}km), membatasi max 5000 bangunan agar browser tidak berat...")
                dataset = dataset.limit(5000)
                
            json_data = geemap.ee_to_geojson(dataset)
            
            # Legend
            legend_dict = {
                'Rendah (< 5m)': COLORS['low'],
                'Sedang (5-10m)': COLORS['medium'],
                'Tinggi (10-15m)': COLORS['high'],
                'Gedung (> 15m)': COLORS['sky']
            }
            m.add_legend(title="Indikasi Ketinggian", legend_dict=legend_dict)

            if json_data and 'features' in json_data and len(json_data['features']) > 0:
                 m.add_geojson(json_data, style_callback=style_function, layer_name="Bangunan")
                 print(f"📌 Menampilkan bangunan di radius {rad_km}km. (Lat: {lat}, Lon: {lon})")
            else:
                 print("⚠️ Tidak ada bangunan ditemukan di area ini.")
            
        except Exception as e:
            print(f"❌ Error memuat data: {e}")
            print("Coba kurangi Radius Area.")

        display(m)

# Bind Button
update_btn.on_click(update_map)

# Tampilkan
display(ui_controls)
display(output_map)

# Run First Time
update_map()